In [20]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

In [21]:
pwd

'/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text'

In [22]:
# cd G:\マイドライブ\ColabNotebooks\Others\graph-2-text

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
cd /content/drive/MyDrive/ColabNotebooks/Others/graph-2-text

/content/drive/MyDrive/ColabNotebooks/Others/graph-2-text


In [25]:
# cd /Users/kosukeaigo/Google Drive/ColabNotebooks/Others/graph-2-text

In [26]:
# !pip uninstall torchtext

In [27]:
# !pip install torchtext

In [28]:
# !pip install torch

In [29]:
import argparse
import os
import glob
import sys

In [30]:
import torch

In [31]:
import onmt.io
import opts

In [32]:
# import importlib
# importlib.reload(onmt.io)

In [33]:
def check_existing_pt_files(opt):
    # We will use glob.glob() to find sharded {train|valid}.[0-9]*.pt
    # when training, so check to avoid tampering with existing pt files
    # or mixing them up.
    for t in ['train', 'valid', 'vocab']:
        pattern = opt.save_data + '.' + t + '*.pt'
        if glob.glob(pattern):
            sys.stderr.write("Please backup exisiting pt file: %s, "
                             "to avoid tampering!\n" % pattern)
            sys.exit(1)

In [34]:
def parse_args():
    parser = argparse.ArgumentParser(
        description='preprocess.py',
        formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    opts.add_md_help_argument(parser)
    opts.preprocess_opts(parser)
    # opt = parser.parse_args()
    opt = parser.parse_args("-train_src data/webnlg/train-webnlg-all-notdelex-src-nodes.txt -train_label data/webnlg/train-webnlg-all-notdelex-src-labels.txt -train_node1 data/webnlg/train-webnlg-all-notdelex-src-node1.txt -train_node2 data/webnlg/train-webnlg-all-notdelex-src-node2.txt -train_tgt data/webnlg/train-webnlg-all-notdelex-tgt.txt -valid_src data/webnlg/dev-webnlg-all-notdelex-src-nodes.txt -valid_label data/webnlg/dev-webnlg-all-notdelex-src-labels.txt -valid_node1 data/webnlg/dev-webnlg-all-notdelex-src-node1.txt -valid_node2 data/webnlg/dev-webnlg-all-notdelex-src-node2.txt -valid_tgt data/webnlg/dev-webnlg-all-notdelex-tgt.txt -save_data data/gcn_exp20210402 -src_vocab_size 5000 -tgt_vocab_size 5000 -data_type gcn ".split())
    torch.manual_seed(opt.seed)
    check_existing_pt_files(opt)
    return opt

In [35]:
def build_save_text_dataset_in_shards(src_corpus, tgt_corpus, fields,
                                      corpus_type, opt):
    '''
    Divide the big corpus into shards, and build dataset separately.
    This is currently only for data_type=='text'.
    The reason we do this is to avoid taking up too much memory due
    to sucking in a huge corpus file.
    To tackle this, we only read in part of the corpus file of size
    `max_shard_size`(actually it is multiples of 64 bytes that equals
    or is slightly larger than this size), and process it into dataset,
    then write it to disk along the way. By doing this, we only focus on
    part of the corpus at any moment, thus effectively reducing memory use.
    According to test, this method can reduce memory footprint by ~50%.
    Note! As we process along the shards, previous shards might still
    stay in memory, but since we are done with them, and no more
    reference to them, if there is memory tight situation, the OS could
    easily reclaim these memory.
    If `max_shard_size` is 0 or is larger than the corpus size, it is
    effectively preprocessed into one dataset, i.e. no sharding.
    NOTE! `max_shard_size` is measuring the input corpus size, not the
    output pt file size. So a shard pt file consists of examples of size
    2 * `max_shard_size`(source + target).
    '''
    corpus_size = os.path.getsize(src_corpus)
    if corpus_size > 10 * (1024**2) and opt.max_shard_size == 0:
        print("Warning. The corpus %s is larger than 10M bytes, you can "
              "set '-max_shard_size' to process it by small shards "
              "to use less memory." % src_corpus)
    if opt.max_shard_size != 0:
        print(' * divide corpus into shards and build dataset separately'
              '(shard_size = %d bytes).' % opt.max_shard_size)
    ret_list = []
    src_iter = onmt.io.ShardedTextCorpusIterator(
                src_corpus, opt.src_seq_length_trunc,
                "src", opt.max_shard_size)
    tgt_iter = onmt.io.ShardedTextCorpusIterator(
                tgt_corpus, opt.tgt_seq_length_trunc,
                "tgt", opt.max_shard_size,
                assoc_iter=src_iter)
    index = 0
    while not src_iter.hit_end():
        index += 1
        dataset = onmt.io.TextDataset(
                fields, src_iter, tgt_iter,
                src_iter.num_feats, tgt_iter.num_feats,
                src_seq_length=opt.src_seq_length,
                tgt_seq_length=opt.tgt_seq_length,
                dynamic_dict=opt.dynamic_dict)

        # We save fields in vocab.pt seperately, so make it empty.
        dataset.fields = []
        pt_file = "{:s}.{:s}.{:d}.pt".format(
                opt.save_data, corpus_type, index)
        print(" * saving %s data shard to %s." % (corpus_type, pt_file))
        torch.save(dataset, pt_file)
        ret_list.append(pt_file)
    return ret_list

In [36]:
def build_save_text_dataset_in_shards_gcn(src_corpus, tgt_corpus,
                                          label_corpus, node1_corpus,
                                          node2_corpus, fields,
                                          corpus_type, opt, mf_corpus=''):
    '''
    Divide the big corpus into shards, and build dataset separately.
    This is currently only for data_type=='text'.
    The reason we do this is to avoid taking up too much memory due
    to sucking in a huge corpus file.
    To tackle this, we only read in part of the corpus file of size
    `max_shard_size`(actually it is multiples of 64 bytes that equals
    or is slightly larger than this size), and process it into dataset,
    then write it to disk along the way. By doing this, we only focus on
    part of the corpus at any moment, thus effectively reducing memory use.
    According to test, this method can reduce memory footprint by ~50%.
    Note! As we process along the shards, previous shards might still
    stay in memory, but since we are done with them, and no more
    reference to them, if there is memory tight situation, the OS could
    easily reclaim these memory.
    If `max_shard_size` is 0 or is larger than the corpus size, it is
    effectively preprocessed into one dataset, i.e. no sharding.
    NOTE! `max_shard_size` is measuring the input corpus size, not the
    output pt file size. So a shard pt file consists of examples of size
    2 * `max_shard_size`(source + target).
    '''
    corpus_size = os.path.getsize(src_corpus)
    if corpus_size > 10 * (1024**2) and opt.max_shard_size == 0:
        print("Warning. The corpus %s is larger than 10M bytes, you can "
              "set '-max_shard_size' to process it by small shards "
              "to use less memory." % src_corpus)
    if opt.max_shard_size != 0:
        print(' * divide corpus into shards and build dataset separately'
              '(shard_size = %d bytes).' % opt.max_shard_size)
    ret_list = []
    src_iter = onmt.io.ShardedTextCorpusIterator(
                src_corpus, opt.src_seq_length_trunc,
                "src", opt.max_shard_size)
    tgt_iter = onmt.io.ShardedTextCorpusIterator(
                tgt_corpus, opt.tgt_seq_length_trunc,
                "tgt", opt.max_shard_size,
                assoc_iter=src_iter)
    label_iter = onmt.io.ShardedTextCorpusIterator(
                label_corpus, opt.src_seq_length_trunc,
                "label", opt.max_shard_size,
                assoc_iter=src_iter)
    node1_iter = onmt.io.ShardedTextCorpusIterator(
        node1_corpus, opt.src_seq_length_trunc,
        "node1", opt.max_shard_size,
                assoc_iter=src_iter)
    node2_iter = onmt.io.ShardedTextCorpusIterator(
        node2_corpus, opt.src_seq_length_trunc,
        "node2", opt.max_shard_size,
                assoc_iter=src_iter)
    if mf_corpus != '':
        mf_iter = onmt.io.ShardedTextCorpusIterator(
            mf_corpus, opt.src_seq_length_trunc,
            "morph", opt.max_shard_size,
            assoc_iter=src_iter)
    else:
        mf_iter = ''
    index = 0
    while not src_iter.hit_end():
        index += 1
        dataset = onmt.io.GCNDataset(
                fields, src_iter, tgt_iter, label_iter,
                node1_iter, node2_iter, mf_iter,
                src_iter.num_feats, tgt_iter.num_feats,
                src_seq_length=opt.src_seq_length,
                tgt_seq_length=opt.tgt_seq_length,
                dynamic_dict=opt.dynamic_dict)

        # We save fields in vocab.pt separately, so make it empty.
        dataset.fields = []
        pt_file = "{:s}.{:s}.{:d}.pt".format(
                opt.save_data, corpus_type, index)
        print(" * saving %s data shard to %s." % (corpus_type, pt_file))
        torch.save(dataset, pt_file)
        ret_list.append(pt_file)
    return ret_list

In [37]:
def build_save_dataset(corpus_type, fields, opt):
    assert corpus_type in ['train', 'valid']
    if corpus_type == 'train':
        src_corpus = opt.train_src
        tgt_corpus = opt.train_tgt
    else:
        src_corpus = opt.valid_src
        tgt_corpus = opt.valid_tgt

    # Currently we only do preprocess sharding for corpus: data_type=='text'.
    if opt.data_type == 'text':
        return build_save_text_dataset_in_shards(
                src_corpus, tgt_corpus, fields,
                corpus_type, opt)
    elif opt.data_type == 'gcn':
        mf_corpus = ''
        if corpus_type == 'train':
            label_corpus = opt.train_label
            node1_corpus = opt.train_node1
            node2_corpus = opt.train_node2
            if opt.train_mf != '':
                mf_corpus = opt.train_mf
        else:
            label_corpus = opt.valid_label
            node1_corpus = opt.valid_node1
            node2_corpus = opt.valid_node2
            if opt.valid_mf != '':
                mf_corpus = opt.valid_mf
        if opt.train_mf != '':
            return build_save_text_dataset_in_shards_gcn(
                src_corpus, tgt_corpus,
                label_corpus, node1_corpus, node2_corpus,
                fields,
                corpus_type, opt, mf_corpus)
        else:
            return build_save_text_dataset_in_shards_gcn(
                src_corpus, tgt_corpus,
                label_corpus, node1_corpus, node2_corpus,
                fields,
                corpus_type, opt)
    # For data_type == 'img' or 'audio', currently we don't do
    # preprocess sharding. We only build a monolithic dataset.
    # But since the interfaces are uniform, it would be not hard
    # to do this should users need this feature.
    dataset = onmt.io.build_dataset(
                fields, opt.data_type, src_corpus, tgt_corpus,
                src_dir=opt.src_dir,
                src_seq_length=opt.src_seq_length,
                tgt_seq_length=opt.tgt_seq_length,
                src_seq_length_trunc=opt.src_seq_length_trunc,
                tgt_seq_length_trunc=opt.tgt_seq_length_trunc,
                dynamic_dict=opt.dynamic_dict,
                sample_rate=opt.sample_rate,
                window_size=opt.window_size,
                window_stride=opt.window_stride,
                window=opt.window)

    # We save fields in vocab.pt separately, so make it empty.
    dataset.fields = []
    pt_file = "{:s}.{:s}.pt".format(opt.save_data, corpus_type)
    print(" * saving %s dataset to %s." % (corpus_type, pt_file))
    torch.save(dataset, pt_file)
    return [pt_file]

In [38]:
def build_save_vocab(train_dataset, fields, opt):
    fields = onmt.io.build_vocab(train_dataset, fields, opt.data_type,
                                 opt.share_vocab,
                                 opt.src_vocab_size,
                                 opt.src_words_min_frequency,
                                 opt.tgt_vocab_size,
                                 opt.tgt_words_min_frequency)

    # Can't save fields, so remove/reconstruct at training time.
    vocab_file = opt.save_data + '.vocab.pt'
    torch.save(onmt.io.save_fields_to_vocab(fields), vocab_file)

In [39]:
def main(opt):
    # opt = parse_args()
    # opt = parser.parse_args("-train_src data/webnlg/train-webnlg-all-notdelex-src-nodes.txt -train_label data/webnlg/train-webnlg-all-notdelex-src-labels.txt -train_node1 data/webnlg/train-webnlg-all-notdelex-src-node1.txt -train_node2 data/webnlg/train-webnlg-all-notdelex-src-node2.txt -train_tgt data/webnlg/train-webnlg-all-notdelex-tgt.txt -valid_src data/webnlg/dev-webnlg-all-notdelex-src-nodes.txt -valid_label data/webnlg/dev-webnlg-all-notdelex-src-labels.txt -valid_node1 data/webnlg/dev-webnlg-all-notdelex-src-node1.txt -valid_node2 data/webnlg/dev-webnlg-all-notdelex-src-node2.txt -valid_tgt data/webnlg/dev-webnlg-all-notdelex-tgt.txt -save_data data/gcn_exp20210402 -src_vocab_size 5000 -tgt_vocab_size 5000 -data_type gcn ".split())
    # opt = easydict.EasyDict({
    #     'train_src':'data/webnlg/train-webnlg-all-notdelex-src-nodes.txt',
    #     'train_label':'data/webnlg/train-webnlg-all-notdelex-src-labels.txt',
    #     'train_node1': 'data/webnlg/train-webnlg-all-notdelex-src-node1.txt',
    #     'train_node2': 'data/webnlg/train-webnlg-all-notdelex-src-node2.txt',
    #     'train_tgt': 'data/webnlg/train-webnlg-all-notdelex-tgt.txt',
    #     'valid_src': 'data/webnlg/dev-webnlg-all-notdelex-src-nodes.txt',
    #     'valid_label': 'data/webnlg/dev-webnlg-all-notdelex-src-labels.txt',
    #     'valid_node1': 'data/webnlg/dev-webnlg-all-notdelex-src-node1.txt',
    #     'valid_node2': 'data/webnlg/dev-webnlg-all-notdelex-src-node2.txt',
    #     'valid_tgt': 'data/webnlg/dev-webnlg-all-notdelex-tgt.txt',
    #     'save_data': 'data/gcn_exp',
    #     'src_vocab_size': 5000,
    #     'tgt_vocab_size': 5000,
    #     'data_type': 'gcn',
    #     'train_mf': '',
    #     'valid_mf': '',
    #     'max_shard_size':0,
    #     'src_seq_length_trunc': 0,
    #     'tgt_seq_length_trunc': 0,
    #     'src_seq_length':128,
    #     'tgt_seq_length':128,
    #     'dynamic_dict':'',
    #     'share_vocab':'',
    #     'src_words_min_frequency':0,
    #     'tgt_words_min_frequency':0
    # })
    print("Extracting features...")
    src_nfeats = onmt.io.get_num_features(opt.data_type, opt.train_src, 'src')
    tgt_nfeats = onmt.io.get_num_features(opt.data_type, opt.train_tgt, 'tgt')
    print(" * number of source features: %d." % src_nfeats)
    print(" * number of target features: %d." % tgt_nfeats)
    print("Building `Fields` object...")
    fields = onmt.io.get_fields(opt.data_type, src_nfeats, tgt_nfeats)
    print("Building & saving training data...")
    print("FIeld"+str(fields))
    train_dataset_files = build_save_dataset('train', fields, opt)
    print("Building & saving vocabulary...")
    build_save_vocab(train_dataset_files, fields, opt)
    print("Building & saving validation data...")
    build_save_dataset('valid', fields, opt)

In [40]:
if __name__ == "__main__":
    opt = parse_args()
    main(opt)

Extracting features...
 * number of source features: 0.
 * number of target features: 0.
Building `Fields` object...
Building & saving training data...
FIeld{'src': <torchtext.legacy.data.field.Field object at 0x7f2068caae90>, 'tgt': <torchtext.legacy.data.field.Field object at 0x7f206a1a3550>, 'src_map': <torchtext.legacy.data.field.Field object at 0x7f2068caa510>, 'alignment': <torchtext.legacy.data.field.Field object at 0x7f2068caab10>, 'indices': <torchtext.legacy.data.field.Field object at 0x7f2068caa610>, 'label': <torchtext.legacy.data.field.Field object at 0x7f2068caa450>, 'node1': <torchtext.legacy.data.field.Field object at 0x7f2068caa710>, 'node2': <torchtext.legacy.data.field.Field object at 0x7f2068caa6d0>, 'morph': <torchtext.legacy.data.field.Field object at 0x7f2068caa850>}
average src size 17.25671196552867 18102
 * saving train data shard to data/gcn_exp20210402.train.1.pt.
Building & saving vocabulary...
 * reloading data/gcn_exp20210402.train.1.pt.
 * tgt vocab size